In [13]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [14]:
print("GPU Available:", tf.config.list_physical_devices('GPU'))

GPU Available: []


In [15]:
data = pd.read_csv("C:/Users/sarth/Downloads/Datasets/sentiment analysis/Twitter_Data.csv")
data.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [16]:
# Ensure column names are correct
text_column = 'clean_text'  # Update if column name differs
category_column = 'category'

In [17]:
# Preprocessing: Extract text and labels
texts = data[text_column].astype(str).tolist()
labels = data[category_column].values

In [18]:
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [19]:
# Tokenization and Padding
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

In [20]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [21]:
X_train_padded = pad_sequences(X_train_seq, maxlen=100, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=100, padding='post', truncating='post')


In [27]:
print("Unique values in y_train:", np.unique(y_train))
print("Unique values in y_test:", np.unique(y_test))

Unique values in y_train: [-2147483648           0           1]
Unique values in y_test: [0 1]


In [24]:
y_train = np.clip(y_train, 0, 2)  # Ensure values are within [0, 2]
y_test = np.clip(y_test, 0, 2)

y_train = y_train.astype(int)
y_test = y_test.astype(int)

y_train = np.nan_to_num(y_train, nan=0).astype(int)
y_test = np.nan_to_num(y_test, nan=0).astype(int)


C:\Users\sarth\AppData\Local\Temp\ipykernel_144\194310400.py:4: RuntimeWarning: invalid value encountered in cast
  y_train = y_train.astype(int)


In [ ]:
# Convert labels to one-hot encoding
y_train_one_hot = tf.keras.utils.to_categorical(y_train, num_classes=3)
y_test_one_hot = tf.keras.utils.to_categorical(y_test, num_classes=3)

IndexError: index -2147483648 is out of bounds for axis 1 with size 3

In [ ]:
# Model Architecture
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=128, input_length=100),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 classes: -1, 0, 1
])

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Display the model summary
model.summary()

In [ ]:
# Train the model
history = model.fit(
    X_train_padded, y_train_one_hot,
    epochs=5,
    batch_size=32,
    validation_data=(X_test_padded, y_test_one_hot)
)

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test_padded, y_test_one_hot)
print(f"Test Accuracy: {test_acc:.2f}")

In [ ]:
# Classification Report
y_pred = model.predict(X_test_padded)
y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(y_test, y_pred_classes, target_names=['Negative', 'Neutral', 'Positive']))